In [4]:
import requests
from requests.auth import HTTPBasicAuth

API_ENDPOINT = "https://<pf-hostname>:9999/pf-admin-api/v1/license"

USERNAME = "Administrator"
PASSWORD = "PF Password"

headers = {
    "Accept": "application/json",
    "X-XSRF-Header": "PingFederate"  # required for CSRF protection
}

try:
    response = requests.get(
        API_ENDPOINT,
        auth=HTTPBasicAuth(USERNAME, PASSWORD),
        headers=headers,
        verify=False
    )

    if response.status_code == 200:
        license_info = response.json()
        print("License Information:")
        print(license_info)
    else:
        print(f"Error {response.status_code}: {response.text}")+

except Exception as e:
    print(f"Failed to fetch license info: {e}")


D:\Agentic-AI\CrewAI-Code\latest_ai_developments\.venv\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'authtesting.likemindscloud.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


License Information:
{'id': '00759624', 'usedConnections': 0, 'tier': 'Free', 'issueDate': '2025-02-03T18:30:00.000Z', 'expirationDate': '2026-02-02T18:30:00.000Z', 'enforcementType': '3', 'version': '12.2', 'product': 'PingFederate', 'organization': 'LikeMinds Consulting Inc', 'gracePeriod': 1, 'licenseGroups': [], 'features': [{'name': 'DeploymentMethod', 'value': 'Traditional'}], 'oauthEnabled': True, 'provisioningEnabled': True, 'wsTrustEnabled': True, 'bridgeMode': False}


# PingFederate Mock Server Analysis

## Current Mock Server Endpoints

Based on the analysis of `pf_agent/simulators/pingfed_mock.py`, the mock server currently implements the following endpoints:

### ✅ **Implemented Endpoints**

1. **License Information Retrieval**
   - **Endpoint**: `GET /{instance}/license`
   - **Response Model**: `LicenseView`
   - **Compliance**: ✅ **COMPLIANT** with PingFederate swagger
   - **Swagger Reference**: `/license` GET operation

2. **License Application/Update**
   - **Endpoint**: `PUT /{instance}/license`
   - **Request Model**: `ApplyLicenseRequest` (base64 encoded license)
   - **Response Model**: `LicenseView`
   - **Compliance**: ✅ **COMPLIANT** with PingFederate swagger
   - **Swagger Reference**: `/license` PUT operation

3. **License Agreement Retrieval**
   - **Endpoint**: `GET /{instance}/license/agreement`
   - **Response Model**: `LicenseAgreement`
   - **Compliance**: ✅ **COMPLIANT** with PingFederate swagger
   - **Swagger Reference**: `/license/agreement` GET operation

4. **License Agreement Acceptance**
   - **Endpoint**: `PUT /{instance}/license/agreement`
   - **Request Model**: `LicenseAgreement`
   - **Response Model**: `LicenseAgreement`
   - **Compliance**: ✅ **COMPLIANT** with PingFederate swagger
   - **Swagger Reference**: `/license/agreement` PUT operation

5. **Cluster Status** (Custom Enhancement)
   - **Endpoint**: `GET /cluster/status`
   - **Response**: JSON with cluster information
   - **Compliance**: ⚠️ **ENHANCEMENT** - Not in original swagger but useful for testing

## Swagger Compliance Analysis

### ✅ **Fully Compliant Areas**

1. **Endpoint Naming**: All license endpoints match the swagger specification exactly
2. **HTTP Methods**: Correct use of GET/PUT methods as defined in swagger
3. **Request/Response Models**: Core license models match swagger definitions
4. **Base Path Structure**: Follows `/pf-admin-api/v1` pattern (minus versioning for simplicity)

### 🔧 **Areas for Enhanced Compliance**

1. **LicenseView Model Completeness**
   - **Current**: 4 core fields (`issuedTo`, `product`, `expiryDate`, `licenseKeyId`)
   - **Swagger**: 20+ fields including:
     - `maxConnections`, `usedConnections`
     - `tier` (FREE, PERPETUAL, SUBSCRIPTION)
     - `issueDate`, `enforcementType`, `version`
     - `organization`, `gracePeriod`, `nodeLimit`
     - `licenseGroups`, `oauthEnabled`, `wsTrustEnabled`
     - `provisioningEnabled`, `bridgeMode`, `features`

2. **Error Response Models**
   - **Missing**: `ApiResult` model for error responses
   - **Swagger**: Defines standard error response format

3. **HTTP Status Codes**
   - **Current**: Basic 200, 400, 404 handling
   - **Swagger**: Includes 422 for validation errors

### 🎯 **Mock Server Strengths**

1. **Multi-Instance Support**: Handles 264+ instances dynamically
2. **Realistic License Parsing**: Extracts EXPIRY, Organization, ID from license content
3. **Base64 Decoding**: Proper handling of encoded license files
4. **Dynamic Route Generation**: Creates routes for all instances automatically
5. **Enterprise Scale**: Supports realistic enterprise deployments

### 📊 **Mock vs Real PingFederate**

| Feature | Mock Server | Real PingFederate | Compliance |
|---------|-------------|-------------------|------------|
| License GET | ✅ 4 fields | ✅ 20+ fields | 🟡 Core compliant |
| License PUT | ✅ Full support | ✅ Full support | ✅ Compliant |
| Agreement GET/PUT | ✅ Basic | ✅ Full | ✅ Compliant |
| Multi-instance | ✅ 264+ instances | ❌ Single instance | 🔄 Enhanced |
| Error handling | 🟡 Basic | ✅ Full | 🟡 Adequate |
| Authentication | ❌ Mocked | ✅ Required | 🟡 Testing only |

## Recommendations for Enhancement

1. **Expand LicenseView Model** to include all swagger fields
2. **Add ApiResult Model** for proper error responses
3. **Implement 422 Status Codes** for validation errors
4. **Add Authentication Simulation** (optional for testing)
5. **Include License Features Array** for advanced license types

## Current Usage & Testing Success

The mock server effectively supports:
- ✅ **264 enterprise instances** across multiple environments
- ✅ **License status monitoring** with OK/WARNING/EXPIRED states
- ✅ **License application testing** with safety checks
- ✅ **Environment filtering** (dev, stage, prod, uat, dr)
- ✅ **Enterprise-scale simulation** for realistic testing

In [ ]:
# Enhanced LicenseView Model for Full Swagger Compliance
from pydantic import BaseModel
from typing import Optional, List
from datetime import datetime

class EnhancedLicenseView(BaseModel):
    """Enhanced LicenseView model matching full PingFederate swagger specification"""
    
    # Core license information (current implementation)
    issuedTo: str = "Ping Identity Corporation"  # name field in swagger
    licenseKeyId: str = "LIC-ABC12345"  # id field in swagger
    product: str = "PingFederate"
    expiryDate: str = "2025-12-31"  # expirationDate in swagger
    
    # Additional swagger fields for full compliance
    maxConnections: Optional[int] = 100
    usedConnections: Optional[int] = 45
    tier: str = "SUBSCRIPTION"  # FREE, PERPETUAL, or SUBSCRIPTION
    issueDate: str = "2024-01-01T00:00:00.000Z"
    enforcementType: str = "7"  # 3-bit binary as decimal
    version: str = "12.1.3.0"
    organization: str = "Enterprise Corp"
    gracePeriod: Optional[int] = 30
    nodeLimit: Optional[int] = 4
    
    # Feature flags
    oauthEnabled: bool = True
    wsTrustEnabled: bool = True
    provisioningEnabled: bool = False
    bridgeMode: bool = False
    
    # Advanced features
    licenseGroups: List[dict] = []
    features: List[dict] = [
        {"name": "OAUTH", "value": "enabled"},
        {"name": "WS_TRUST", "value": "enabled"}
    ]

# Example API Response Structure
class ApiResult(BaseModel):
    """Standard error response model from swagger"""
    message: str
    validationErrors: Optional[List[dict]] = None

# Demo: How current vs enhanced would look
print("=== CURRENT MOCK RESPONSE ===")
current = {
    "issuedTo": "Ping Identity Corporation",
    "product": "PingFederate", 
    "expiryDate": "2025-12-31",
    "licenseKeyId": "LIC-ABC12345"
}
print(f"Fields: {len(current)}")
for key, value in current.items():
    print(f"  {key}: {value}")

print("\n=== ENHANCED SWAGGER-COMPLIANT RESPONSE ===")
enhanced = EnhancedLicenseView()
enhanced_dict = enhanced.dict()
print(f"Fields: {len(enhanced_dict)}")
for key, value in enhanced_dict.items():
    print(f"  {key}: {value}")

print(f"\nCompliance improvement: {len(enhanced_dict)} vs {len(current)} fields")
print("Status: Current mock covers core functionality ✅")
print("Enhancement: Could add remaining fields for 100% swagger compliance 🎯")